In [12]:
'''
Load the training dataframe
Sample for 10000 rows
Convert the rows to appropriate format for fast DTW 
Pairwise DTW on all samples - 10^6 copmutations, ~10^-3
'''

'\nLoad the training dataframe\nSample for 10000 rows\nConvert the rows to appropriate format for fast DTW \nPairwise DTW on all samples - 10^6 copmutations, ~10^-3\n'

In [13]:
import sklearn.metrics.pairwise as skm
import sklearn.preprocessing as skp
import pandas as pd
import numpy as np
import fastdtw
from timeit import default_timer as timer
import sklearn.cluster as skc

In [14]:
def dtw(arr1, arr2):
    error, _ = fastdtw.fastdtw(arr1, arr2, radius=1, dist=2)
    return error

In [23]:
rawData_df = pd.read_csv('train_1.csv').sample(n=200)
# Fill all NaN with zeros
rawData_df.fillna(value=0.0,inplace=True)
rawData_df.drop('Page',axis=1,inplace=True)

# Could also scale [0,1/N] or z-normalize
# scaled_df = skp.MinMaxScaler(feature_range=(0, 1), copy=True).fit_transform(rawData_df)

In [30]:
# Write some sample rows to file
rawData_df.to_csv(path_or_buf='Sampled_Rows.txt',header=False, sep=' ',index=False, index_label=False, line_terminator=' ', na_rep=0.0)

In [31]:
sample_df = rawData_df.sample(n=1)
sample_df.to_csv(path_or_buf='Query_Row.txt',header=False, sep=' ',index=False, index_label=False,line_terminator=' ')

In [38]:
rawData_df.head(42)

,2015-07-01,2015-07-02,2015-07-03,2015-07-04,2015-07-05,2015-07-06,2015-07-07,2015-07-08,2015-07-09,2015-07-10,...,2016-12-22,2016-12-23,2016-12-24,2016-12-25,2016-12-26,2016-12-27,2016-12-28,2016-12-29,2016-12-30,2016-12-31
136215,4.0,2.0,4.0,4.0,3.0,2.0,2.0,7.0,1.0,5.0,...,33.0,221.0,139.0,77.0,52.0,67.0,25.0,45.0,48.0,35.0
48457,9.0,39.0,16.0,8.0,7.0,13.0,3.0,12.0,16.0,10.0,...,9.0,11.0,10.0,8.0,4.0,25.0,8.0,4.0,19.0,7.0
54607,54.0,44.0,59.0,52.0,65.0,87.0,48.0,57.0,43.0,62.0,...,220.0,197.0,291.0,302.0,308.0,267.0,250.0,295.0,318.0,358.0
2476,1.0,1.0,6.0,6.0,4.0,4.0,0.0,5.0,2.0,18.0,...,9.0,5.0,4.0,12.0,8.0,8.0,17.0,8.0,8.0,6.0
57273,45.0,35.0,46.0,50.0,67.0,49.0,53.0,34.0,58.0,34.0,...,63.0,40.0,77.0,62.0,31.0,55.0,49.0,59.0,52.0,53.0
117896,99.0,95.0,1942.0,511.0,199.0,202.0,225.0,356.0,159.0,200.0,...,143.0,245.0,274.0,288.0,237.0,10983.0,1559.0,639.0,361.0,350.0
110587,4.0,2.0,2.0,4.0,2.0,2.0,4.0,2.0,3.0,7.0,...,6.0,5.0,4.0,1.0,0.0,8.0,0.0,2.0,1.0,6.0
91987,157.0,146.0,149.0,102.0,210.0,102.0,104.0,151.0,199.0,177.0,...,574.0,158.0,160.0,295.0,168.0,130.0,150.0,136.0,140.0,320.0
61644,939.0,611.0,708.0,636.0,669.0,689.0,768.0,891.0,670.0,538.0,...,427.0,446.0,400.0,382.0,418.0,446.0,477.0,519.0,621.0,385.0
7017,68.0,192.0,75.0,109.0,106.0,136.0,90.0,83.0,60.0,66.0,...,81.0,67.0,61.0,67.0,88.0,82.0,75.0,89.0,86.0,76.0


In [33]:
sample_df

,2015-07-01,2015-07-02,2015-07-03,2015-07-04,2015-07-05,2015-07-06,2015-07-07,2015-07-08,2015-07-09,2015-07-10,...,2016-12-22,2016-12-23,2016-12-24,2016-12-25,2016-12-26,2016-12-27,2016-12-28,2016-12-29,2016-12-30,2016-12-31
65884,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,145.0,109.0,66.0,96.0,74.0,123.0,119.0,134.0,134.0,94.0


In [66]:
# n_jobs = -1 to turn on parallel
start = timer()
K = skm.pairwise_kernels(scaled_df, metric=dtw, n_jobs=1)
end = timer()
print(end - start)      

# FASTDTW ON SCALE 4E-4 PER PAIR
# AT 10^3 SAMPLES, 10^6 ELS, SO 10^2 SECONDS. Tolerance for production up to 10^4 elements 
# several hours of runtime

198.1589615730336


In [67]:
## Run Hierarchical clustering on the matrix 
cluster = skc.AgglomerativeClustering(n_clusters=10, affinity='euclidean', connectivity=K, compute_full_tree=True, linkage='ward')
labels = cluster.fit_predict(K)

array([3, 8, 6, 7, 3, 1, 4, 8, 9, 1, 3, 6, 6, 8, 5, 3, 8, 9, 4, 8, 4, 9, 1,
       3, 3, 4, 8, 2, 9, 3, 4, 6, 2, 8, 3, 9, 8, 8, 1, 9, 6, 3, 4, 6, 6, 9,
       9, 4, 3, 1, 6, 4, 9, 6, 6, 3, 3, 9, 3, 4, 8, 4, 6, 3, 9, 2, 9, 1, 9,
       3, 8, 3, 3, 1, 6, 1, 0, 2, 3, 0, 6, 2, 6, 0, 4, 3, 3, 9, 6, 6, 0, 6,
       6, 1, 3, 8, 6, 8, 8, 6])